<div class="row">
  <div class="col-sm-8">
     <center>
        <h1>
        IGR 204 - Visualisation<br>
        Mini-projet (Prénoms en France)<br>
        PARTIE 2 - Evolution dans l'espace'
            <br>
            <br>
            <center><img src="https://s3-eu-west-1.amazonaws.com/assets.atout-on-line.com/images/ingenieur/Fiches_Ecoles/telecom_paristech.jpg" alt="drawing" width="170"/></center>
        <hr class="bg-light" />
            <i>Léa PAPILLON, Pierrick LEROY, Yann RODRIGUEZ, Ramzi KHALIFA, Alexandre LE BRIS<br></i>
            <i>MS IA - 21/22<br></i>
        </h1>
    </center>
  </div>
  </div>
</div>

In [2]:
import geopandas as gpd
import altair as alt
import pandas as pd
import panel as pn

# Reading our names data

Now, let's read in our dataset.  The exported data is in CSV format, but with a `;` separator instead of commas.  The INSEE data collapses rare names or where department-level information has been elided (presumably to protect individuals with uncommon names or who were one of the only ones born with that name in a given year).  We'll strip those out.

In [30]:
df = pd.read_csv('data/dpt2020.csv', sep=';')
df.drop(df[df.preusuel == '_PRENOMS_RARES'].index, inplace=True)
df.drop(df[df.dpt == 'XX'].index, inplace=True)

df.annais = df.annais.astype(int)

df.sample(5)

,sexe,preusuel,annais,dpt,nombre
3510418,2,SANAA,2018,33,5
1059183,1,LUDOVIC,1994,48,5
363634,1,DAVID,1980,13,191
1471596,1,RUBEN,2009,974,11
1622487,1,URBAIN,1916,59,3


# Loading map data

Next, let's load some map data of regions in France using `geopandas`.  These map data come from the [INSEE] and [IGN] and were processed into the `geojson` format we'll need to work with by [Grégoire David].  Here's the [github] repository.

In this example, we'll work with the simplified departments tiles for the Hexagon, but that repository contains higher-resolution versions, the DOM-TOM, and more.

[Grégoire David]: https://gregoiredavid.fr
[INSEE]: http://www.insee.fr/fr/methodes/nomenclatures/cog/telechargement.asp
[IGN]: https://geoservices.ign.fr/adminexpress
[github]: https://github.com/gregoiredavid/france-geojson/

In [31]:
depts = gpd.read_file('data/departements-version-simplifiee.geojson')

depts.sample(5)

,code,nom,geometry
36,36,Indre,"POLYGON ((1.32667 47.18623, 1.40143 47.21245, ..."
73,73,Savoie,"POLYGON ((6.80252 45.77837, 6.80842 45.72515, ..."
41,41,Loir-et-Cher,"POLYGON ((0.84122 48.10306, 0.87589 48.10944, ..."
2,03,Allier,"POLYGON ((3.03207 46.79491, 3.04907 46.75808, ..."
86,86,Vienne,"POLYGON ((-0.10212 47.06480, -0.09806 47.09135..."


Notice how `depts` is a geopandas dataframe.  We'll use it just as a regular `pandas` dataframe, but it includes the geometry info we need to be able to draw those regions when we pass them into Altair.  We just need to make sure that when we work with our data, we keep them in a geopandas dataframe and not a plain dataframe if we want to draw the departments.

In the next cell, notice how we do a right-merge to bring in department data into names.  We do this as a merge on `depts` because we need a geopandas dataframe.  Remember, `depts` is a geopandas dataframe, while `names` is a regular dataframe.  If we did a left merge on `names`, we'd end up with a regular pandas dataframe. After this merge, both `names` and `depts` will be geopandas dataframes.

**Hint:** Be careful when you do your data joins here.  It's easy to accidentally merge the wrong way to accidentally create a _much bigger_ dataset.

In [12]:
# Keep a reference around to the plain pandas dataframe, without geometry data, just in case
just_names = df
df = depts.merge(df, how='right', left_on='code', right_on='dpt')
df.sample(5)

,code,nom,geometry,sexe,preusuel,annais,dpt,nombre
777474,42,Loire,"POLYGON ((3.89953 46.27591, 3.90940 46.25773, ...",1,JEAN-CLAUDE,1980,42,3
1286273,60,Oise,"POLYGON ((1.78384 49.75831, 1.80898 49.75433, ...",1,PASCAL,1950,60,16
3017981,75,Paris,"POLYGON ((2.41634 48.84924, 2.46226 48.84254, ...",2,MARIE-CÉCILE,1983,75,6
2463307,26,Drôme,"POLYGON ((4.80049 45.29836, 4.85880 45.30895, ...",2,GEORGETTE,1904,26,10
85665,74,Haute-Savoie,"POLYGON ((6.80252 45.77837, 6.75551 45.76635, ...",1,ALLAN,2004,74,6


# Show a name over all years

Now we'll choose a name to show across all years.  To that, we'll group all of the names in a department together (squashing the years together) and use the sum.

In [6]:
grouped = df.groupby(['dpt', 'preusuel', 'sexe'], as_index=False).sum()
grouped = depts.merge(grouped, how='right', left_on='code', right_on='dpt') # Add geometry data back in
grouped

,code,nom,geometry,dpt,preusuel,sexe,nombre
0,01,Ain,"POLYGON ((4.78021 46.17668, 4.79458 46.21832, ...",01,AARON,1,160
1,01,Ain,"POLYGON ((4.78021 46.17668, 4.79458 46.21832, ...",01,ABBY,2,3
2,01,Ain,"POLYGON ((4.78021 46.17668, 4.79458 46.21832, ...",01,ABDALLAH,1,7
3,01,Ain,"POLYGON ((4.78021 46.17668, 4.79458 46.21832, ...",01,ABDEL,1,3
4,01,Ain,"POLYGON ((4.78021 46.17668, 4.79458 46.21832, ...",01,ABDELKADER,1,3
...,...,...,...,...,...,...,...
239574,NaN,NaN,None,974,ÉSAÏE,1,3
239575,NaN,NaN,None,974,ÉTHAN,1,53
239576,NaN,NaN,None,974,ÉTIENNE,1,3
239577,NaN,NaN,None,974,ÉVA,2,32


In [7]:
grouped['ratio'] = grouped.nombre/grouped.groupby('dpt').nombre.transform('sum')

Now let's pick a name and check out how it's distribution over the last 120 years across Metropolitan France.  In this example, I choose the name “Lucien,” which I rather like for some reason.

In [29]:
def plot_loc(df_selection):
    chart = alt.Chart(df_selection).mark_geoshape(stroke='white').encode(
                tooltip=['nom', 'code', 'nombre'],
                color='ratio',
            ).properties(width=800, height=600)
    
    return chart

In [27]:
def select_data_loc(annee_range, prenom_regex='paul'):
    
    filter_annee = ((df.annais >= int(annee_range[0])) &
                    (df.annais <= int(annee_range[1])))
    
    df_selection = df[filter_annee]
    
    df_selection = df_selection.groupby(['dpt', 'preusuel', 'sexe'], as_index=False).sum()
    df_selection = depts.merge(df_selection, how='right', left_on='code', right_on='dpt') 

    df_selection['ratio'] = df_selection.nombre/df_selection.groupby('dpt').nombre.transform('sum')

    filter_prenom = df_selection.preusuel.str.contains(prenom_regex.upper(), regex=True, na=False)
    df_selection = df_selection[filter_prenom]

    return plot_loc(df_selection)

In [28]:
text = ("## Discover name trends by location !\n")

name = pn.widgets.TextInput(name='Regex Prénom (ajouter $ pour prenom uniquement)',
                            value='paul$',
                            placeholder='Taper la requête (regex possible)')

annee_range = pn.widgets.IntRangeSlider(name='Année(s)',
                                        start=1900, end=2020,
                                        value=(1900, 2020),
                                        step=1)

widget = pn.Column(name,
                   annee_range)

reactive_selection = pn.bind(select_data_loc,
                            annee_range,
                            name)

plot = pn.Column(text,
                 widget,
                 reactive_selection)

plot.show();

Launching server at http://localhost:57535
